In [11]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tqdm

In [12]:
def prepare_data(input_ds, subtask):
    # First Step
    # change datatype to float32
    ds = input_ds.map(lambda img, target: (tf.cast(img, tf.float32), target))
    # flatten images
    ds = ds.map(lambda img, target: (tf.reshape(img, (-1,)), target))
    # normalize image values
    ds = ds.map(lambda img, target: ((img/128.)-1.0, target))

    # Second Step
    zipped_ds = tf.data.Dataset.zip((ds.shuffle(1000), ds.shuffle(1000)))
    # a + b >= 5 ?
    if subtask == 1:
        zipped_ds = zipped_ds.map(lambda x1, x2:
                                 (x1[0], x2[0], x1[1] + x2[1] >= 5))
    # a - b = ?
    elif subtask == 2:
        zipped_ds = zipped_ds.map(lambda x1, x2:
                                 (x1[0], x2[0], x1[1] - x2[1]))
    else:
        print(f"{subtask} is not a valid subtask!")
        exit(-1)

    # Third Step
    # shuffle the data
    zipped_ds = zipped_ds.shuffle(1000)
    # create batches
    zipped_ds = zipped_ds.batch(32)
    # prefetch
    zipped_ds = zipped_ds.prefetch(tf.data.AUTOTUNE)

    return zipped_ds


In [13]:
def get_mnist_data(subtask):
    train_dataset, test_dataset = tfds.load("mnist", split=["train", "test"],
                                        as_supervised=True)

    train_ds = prepare_data(train_dataset, subtask)
    test_ds = prepare_data(test_dataset, subtask)

    return (train_ds, test_ds)

In [14]:
# load dataset
train_dataset, test_dataset = tfds.load("mnist", split=["train", "test"],
                                        as_supervised=True)

# transform dataset
# subtask 1
train_ds_t1 = prepare_data(train_dataset, 1)
test_ds_t1 = prepare_data(test_dataset, 1)

# subtask 2
train_ds_t2 = prepare_data(train_dataset, 2)
test_ds_t2 = prepare_data(test_dataset, 2)



In [15]:
class MNISTMath(tf.keras.Model):
    def __init__(self, subtask, optimizer):
        super().__init__()
        self.optimizer = optimizer

        self.dense1 = tf.keras.layers.Dense(32, activation="relu")
        self.dense2 = tf.keras.layers.Dense(32, activation="relu")

        # a + b >= 5 ?
        if subtask == 1:
            self.metrics_list = [tf.keras.metrics.Mean(name="loss"),
                                 tf.keras.metrics.BinaryAccuracy()]
            self.loss_function = tf.keras.losses.BinaryCrossentropy()
            self.out_layer = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)
        # a + b = ?
        elif subtask == 2:
            self.metrics_list = [tf.keras.metrics.Mean(name="loss"),
                                 tf.keras.metrics.MeanSquaredError()]
            self.loss_function = tf.keras.losses.MeanSquaredError()
            self.out_layer = tf.keras.layers.Dense(1, activation=None)
        else:
            print(f"{subtask} is not a valid subtask!")
            exit(-1)



    def call(self, images):
        img1, img2 = images

        img1_x = self.dense1(img1)
        img1_x = self.dense2(img1_x)

        img2_x = self.dense1(img2)
        img2_x = self.dense2(img2_x)

        combined_x = tf.concat([img1_x, img2_x], axis=1)

        return self.out_layer(combined_x)

    
    @property
    def metrics(self):
        return self.metrics_list

    
    def reset_metrics(self):
        for metric in self.metrics:
            metric.reset_states()

    
    def train_step(self, data):
        img1, img2, label = data

        with tf.GradientTape() as tape:
            output = self((img1, img2))
            loss = self.loss_function(label, output)
        
        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, 
                                           self.trainable_variables))

        self.metrics[0].update_state(loss)
        self.metrics[1].update_state(output, label)

        return {m.name: m.result() for m in self.metrics}

    
    def test_step(self, data):
        img1, img2, label = data
        output = self((img1, img2))
        loss = self.loss_function(label, output)

        self.metrics[0].update_state(loss)
        self.metrics[1].update_state(output, label)

        return {f"val_{m.name}": m.result() for m in self.metrics}

In [16]:
def train_model(model, subtask):
    train_ds, test_ds = get_mnist_data(subtask)

    for e in range(10):
        for data in tqdm.tqdm(train_ds):
            metrics = model.train_step(data)

        for name, value in metrics.items():
            print(f"Epoch {e}: {name}: {value}")
        model.reset_metrics()

        for data in test_ds:
            metrics = model.test_step(data)
        
        for name, value in metrics.items():
            print(f"Epoch {e}: {name}: {value}")
        model.reset_metrics()        



In [10]:
optimizer = tf.keras.optimizers.Adam()
# optimizer = tf.keras.optimizers.SGD()

model1 = MNISTMath(1, optimizer)
model2 = MNISTMath(2, optimizer)

train_model(model1, 1)
train_model(model2, 2)

100%|██████████| 1875/1875 [00:29<00:00, 63.43it/s]


Epoch 0: loss: 0.2126404494047165
Epoch 0: binary_accuracy: 0.0
Epoch 0: val_loss: 0.15290650725364685
Epoch 0: val_binary_accuracy: 0.0


100%|██████████| 1875/1875 [00:27<00:00, 68.42it/s]


Epoch 1: loss: 0.1497514545917511
Epoch 1: binary_accuracy: 0.0013000000035390258
Epoch 1: val_loss: 0.1421184539794922
Epoch 1: val_binary_accuracy: 0.006500000134110451


100%|██████████| 1875/1875 [00:25<00:00, 73.57it/s]


...

Epoch 24: loss: 0.07812958210706711
Epoch 24: binary_accuracy: 0.303849995136261
Epoch 24: val_loss: 0.10552582144737244
Epoch 24: val_binary_accuracy: 0.2890999913215637


100%|██████████| 1875/1875 [00:26<00:00, 70.42it/s]


Epoch 25: loss: 0.0817280113697052
Epoch 25: binary_accuracy: 0.2893333435058594
Epoch 25: val_loss: 0.10069206357002258
Epoch 25: val_binary_accuracy: 0.2985999882221222


 32%|███▏      | 603/1875 [00:09<00:19, 66.70it/s]


KeyboardInterrupt: 